Lets simulate a CRN. We can run a loop for a set number of iterations where each iteration the sensor network can allocate channels to a primary user and secondary users can sense(query) and use unused channels, how they use the channel will affect their trustValue as assessed by other users. Every some number of iterations all the users will use trimessage communication to synchronize their trust values. Each iteration a fraction of secondary users can also try to be malicious by giving other users lower trust scores, giving themself a high trust score, or lying about whether the primary user is in the channel. Since we don't have a data fusion center, other users will have to "catch" these lies and adjust trust scores accordingly.

In [2]:
class SensorNetwork:
    users = [] #devices in the network
    numChannels = 1 #available channels
    
    def __init__(self, userlist, channels):
        self.users = userlist
        self.numChannels = channels
    
    def allocateChannel(self, val):
        #go through users and determine which deserves a channel
        if val < self.numChannels:
            okay = True
            
    def synchronizeTrust(self):
        for user in users:
            user.synchronizeTrust()
    

In [4]:
class User:
    id_value = 0 # this user's id for distinguishability
    channelAllocated = False
    timeSinceAllocation = 0 # elapsed time since this device used a channel 
    
    def __init__(self, id_val):
        self.id_value = id_val
    
class SecondaryUser(User): 
    trustValue = 0
    users = [] #other users that this device is knows of
    trustValues = [] #perceived trust values of other users
    
    def synchronizeTrust(self):
        i = 0
        for user in users:
            #compare two trust values
            dif = self.users[i].trustValue - self.trustValues[i]
            i+=1
            
        


In [5]:
def initializeUsers(numUsers):
    users = []
    # Add 1 primary User
    # Add Secondary Users to network
    for i in range(numUsers):
        users.append(SecondaryUser(i))
    return users


users = initializeUsers(5)
network = SensorNetwork(users, 3)
        